# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find a recommended location for a given type of restaurant. Specifically, this report will be targeted to stakeholders interested in opening a specific type of restaurant in Los Angeles county.

Since there are lots of restaurants in LA we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Italian restaurants in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* the region vs city mapping will be done using web scraping from a web page published by LA Times 
* coordinate of LA will be obtained using some open data sources made available by the city of Los Angeles

In [1]:
!pip install BeautifulSoup4

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191208050232-0001
KERNEL_ID = 6b090f74-29f5-43c1-b51b-3be480400bde
  Using cached https://files.pythonhosted.org/packages/3b/c8/a55eb6ea11cd7e5ac4bacdf92bac4693b90d3ba79268be16527555e186f0/beautifulsoup4-4.8.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/81/94/03c0f04471fc245d08d0a99f7946ac228ca98da4fa75796c507f61e688c2/soupsieve-1.9.5-py2.py3-none-any.whl
Target directory /home/spark/shared/user-libs/python3.6/soupsieve-1.9.5.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/soupsieve already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/beautifulsoup4-4.8.1.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/bs4 already exists. Specify --upgrade to force replacement.


In [2]:
!pip install lxml

    100% |################################| 5.8MB 2.0MB/s eta 0:00:01


In [3]:
!pip install html5lib

    100% |################################| 122kB 4.4MB/s eta 0:00:01
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [5]:
#load URLs we want to scrape into an array
BASE_URL = [
'http://maps.latimes.com/neighborhoods/neighborhood/list/'
]

In [6]:
postcode_members = []
#Loop through our URLs we loaded above
for b in BASE_URL:
    html = requests.get(b).text
    soup = BeautifulSoup(html, "html.parser")
    #identify table we want to scrape
    #wiki_table = soup.find('table', {"class" : "dataTable"})
    Neighbor_table = soup.find('tbody')
    #try clause to skip any companies with missing/empty board member tables
    try:
        #loop through table, grab each of the 4 columns shown (try one of the links yourself to see the layout)
        for row in Neighbor_table.find_all('tr'):
            cols = row.find_all('td')
            #print(len(cols))
            if len(cols) == 2:
                postcode_members.append((b, cols[0].text.strip(), cols[1].text.strip()))                
    except: pass  

In [7]:
#convert output to new array, check length
postcode_array = np.asarray(postcode_members)
len(postcode_array)

272

In [8]:
#convert new array to dataframe
df = pd.DataFrame(postcode_array)
df.columns = ['URL', 'Postal_City', 'Region']

In [9]:
df.head()

,URL,Postal_City,Region
0,http://maps.latimes.com/neighborhoods/neighbor...,Acton,Antelope Valley
1,http://maps.latimes.com/neighborhoods/neighbor...,Adams-Normandie,South L.A.
2,http://maps.latimes.com/neighborhoods/neighbor...,Agoura Hills,Santa Monica Mountains
3,http://maps.latimes.com/neighborhoods/neighbor...,Agua Dulce,Northwest County
4,http://maps.latimes.com/neighborhoods/neighbor...,Alhambra,San Gabriel Valley


In [10]:
df.drop(['URL'], axis=1)

,Postal_City,Region
0,Acton,Antelope Valley
1,Adams-Normandie,South L.A.
2,Agoura Hills,Santa Monica Mountains
3,Agua Dulce,Northwest County
4,Alhambra,San Gabriel Valley
5,Alondra Park,South Bay
6,Altadena,Verdugos
7,Angeles Crest,Angeles Forest
8,Arcadia,San Gabriel Valley
9,Arleta,San Fernando Valley


In [11]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3aca6f1357a04ba1b775efaed124cf94 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fb_KAjgaj7pEa9OGuwLI7Nb8pA00X4P-RTdoX1HfS1Tc',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3aca6f1357a04ba1b775efaed124cf94.get_object(Bucket='default-donotdelete-pr-az9ixstrcuox9j',Key='ZIP_Codes_and_Postal_Cities.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,ZIP Code,Postal City 1,Postal City 2,Postal City 3,Not Acceptable 1,Not Acceptable 2,Not Acceptable 3,Location,Zip,Latitude,Longitude
0,90713,Lakewood,NaN,NaN,NaN,NaN,NaN,"90713(33.84871142900005, -118.11357922799999)",90713,33.848711,-118.113579
1,91306,Winnetka,NaN,NaN,NaN,NaN,NaN,"91306(34.208404020000046, -118.57593995299999)",91306,34.208404,-118.575940
2,90002,Los Angeles,NaN,NaN,NaN,NaN,NaN,"90002(33.94895070600006, -118.24697958699994)",90002,33.948951,-118.246980
3,90506,Torrance,NaN,NaN,NaN,NaN,NaN,"90506(33.88535286100006, -118.32659746799999)",90506,33.885353,-118.326598
4,90069,West Hollywood,Los Angeles,NaN,NaN,NaN,NaN,"90069(34.08940300900008, -118.37978902499998)",90069,34.089403,-118.379789


In [12]:
df_data_1[df_data_1['Postal City 1'] == "Long Beach"]

,ZIP Code,Postal City 1,Postal City 2,Postal City 3,Not Acceptable 1,Not Acceptable 2,Not Acceptable 3,Location,Zip,Latitude,Longitude
16,90822,Long Beach,NaN,NaN,NaN,NaN,NaN,"90822(33.79831371000006, -118.15136774099994)",90822,33.798314,-118.151368
17,90840,Long Beach,NaN,NaN,Ca State Univ Long Beach,NaN,NaN,"90840(33.77534166600003, -118.11346625799996)",90840,33.775342,-118.113466
39,90802,Long Beach,NaN,NaN,NaN,NaN,NaN,"90802(33.76935424900006, -118.184528823)",90802,33.769354,-118.184529
77,90808,Long Beach,NaN,NaN,NaN,NaN,NaN,"90808(33.82496162900003, -118.11531722999996)",90808,33.824962,-118.115317
107,90806,Long Beach,NaN,NaN,NaN,NaN,NaN,"90806(33.80148122000003, -118.18919970399998)",90806,33.801481,-118.189200
114,90846,Long Beach,NaN,NaN,NaN,NaN,NaN,"90846(33.81634113800004, -118.15870645499996)",90846,33.816341,-118.158706
146,90804,Long Beach,NaN,NaN,East Long Beach,NaN,NaN,"90804(33.78173317000005, -118.15235840199995)",90804,33.781733,-118.152358
179,90815,Long Beach,NaN,NaN,NaN,NaN,NaN,"90815(33.79356027400007, -118.12236917399997)",90815,33.793560,-118.122369
194,90810,Long Beach,Carson,NaN,Cabrillo,Dominguez,NaN,"90810(33.813560335000034, -118.21653847799996)",90810,33.813560,-118.216538
210,90807,Long Beach,NaN,NaN,Bixby Knolls,NaN,NaN,"90807(33.83250021400005, -118.181317864)",90807,33.832500,-118.181318


In [13]:
df_data_temp1 = df_data_1.rename(columns={"ZIP Code": "Zip_Code", "Postal City 1": "Postal_City"})

In [14]:
df_data = df_data_temp1.drop(columns=['Zip', 'Postal City 2', 'Postal City 3', 'Not Acceptable 1', 'Not Acceptable 2', 'Not Acceptable 3', 'Location'])

In [15]:
df_data.head()

,Zip_Code,Postal_City,Latitude,Longitude
0,90713,Lakewood,33.848711,-118.113579
1,91306,Winnetka,34.208404,-118.575940
2,90002,Los Angeles,33.948951,-118.246980
3,90506,Torrance,33.885353,-118.326598
4,90069,West Hollywood,34.089403,-118.379789


In [16]:
df_data.shape

(370, 4)

In [17]:
df_merge = pd.merge(df_data, df, on='Postal_City', how='left')

In [18]:
df_merge.shape

(370, 6)

In [19]:
df_merge.head()

,Zip_Code,Postal_City,Latitude,Longitude,URL,Region
0,90713,Lakewood,33.848711,-118.113579,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
1,91306,Winnetka,34.208404,-118.575940,http://maps.latimes.com/neighborhoods/neighbor...,San Fernando Valley
2,90002,Los Angeles,33.948951,-118.246980,NaN,NaN
3,90506,Torrance,33.885353,-118.326598,http://maps.latimes.com/neighborhoods/neighbor...,South Bay
4,90069,West Hollywood,34.089403,-118.379789,http://maps.latimes.com/neighborhoods/neighbor...,Central L.A.


In [20]:
df_merge[df_merge['Region'].isna()]

,Zip_Code,Postal_City,Latitude,Longitude,URL,Region
2,90002,Los Angeles,33.948951,-118.246980,NaN,NaN
6,90064,Los Angeles,34.035412,-118.432177,NaN,NaN
14,90006,Los Angeles,34.050102,-118.292689,NaN,NaN
15,90018,Los Angeles,34.030772,-118.316869,NaN,NaN
21,90037,Los Angeles,34.003494,-118.287097,NaN,NaN
24,90035,Los Angeles,34.054282,-118.383856,NaN,NaN
27,90620,Buena Park,33.842780,-118.012349,NaN,NaN
29,90045,Los Angeles,33.961521,-118.393450,NaN,NaN
33,93553,Pearblossom,34.470473,-117.899379,NaN,NaN
35,90033,Los Angeles,34.048564,-118.211520,NaN,NaN


In [21]:
df_merge.Region.fillna(df_merge.Postal_City, inplace=True)

In [22]:
df_merge.head(10)

,Zip_Code,Postal_City,Latitude,Longitude,URL,Region
0,90713,Lakewood,33.848711,-118.113579,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
1,91306,Winnetka,34.208404,-118.575940,http://maps.latimes.com/neighborhoods/neighbor...,San Fernando Valley
2,90002,Los Angeles,33.948951,-118.246980,NaN,Los Angeles
3,90506,Torrance,33.885353,-118.326598,http://maps.latimes.com/neighborhoods/neighbor...,South Bay
4,90069,West Hollywood,34.089403,-118.379789,http://maps.latimes.com/neighborhoods/neighbor...,Central L.A.
5,91361,Westlake Village,34.151380,-118.835697,http://maps.latimes.com/neighborhoods/neighbor...,Santa Monica Mountains
6,90064,Los Angeles,34.035412,-118.432177,NaN,Los Angeles
7,91308,West Hills,34.200492,-118.629227,http://maps.latimes.com/neighborhoods/neighbor...,San Fernando Valley
8,91702,Azusa,34.127122,-117.906210,http://maps.latimes.com/neighborhoods/neighbor...,San Gabriel Valley
9,91104,Pasadena,34.167000,-118.127588,http://maps.latimes.com/neighborhoods/neighbor...,Verdugos


In [23]:
df_merge[df_merge['Region'].isna()]

,Zip_Code,Postal_City,Latitude,Longitude,URL,Region


In [24]:
#Before we get the data and start exploring it, let's download all the dependencies that we will need.
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/spark/shared/conda/envs/python3.6

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

The followin

In [25]:
#Use geopy library to get the latitude and longitude values of Los Angeles.
#In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent la_explorer, as shown below.
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [26]:
#Create a map of Los Angeles County with Cities and Zipcode superimposed on top.
# create map of Los Angeles County using latitude and longitude values
print("latitude = ", latitude)
print("longitude = ", longitude)
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postalcity, zipcode in zip(df_merge['Latitude'], df_data['Longitude'], df_data['Postal_City'], df_data['Zip_Code']):
    label = '{}, {}'.format(postalcity, zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

latitude =  34.0536909
longitude =  -118.2427666


In [130]:
#Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#Define Foursquare Credentials and Version
#Define Foursquare Credentials and Version
#Make sure that you have created a Foursquare developer account and have your credentials handy
CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [131]:
#Let's explore the first city in our dataframe.
#Get the city's name and Zip code.
df_merge.loc[0, 'Postal_City']
df_merge.loc[0, 'Zip_Code']

90713

In [132]:
#Get the city's latitude and longitude
city_latitude = df_merge.loc[0, 'Latitude'] # neighborhood latitude value
city_longitude = df_merge.loc[0, 'Longitude'] # neighborhood longitude value

city_name = df_merge.loc[0, 'Postal_City'] # neighborhood name
#city_zip = df_data.loc[0, 'Zip_Code'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Lakewood are 33.84871143, -118.11357920000002.


In [133]:
#Now, let's get the top 100 venues that are in Lakewood 90713 within a radius of 500 meters.
#First, let's create the GET request URL. Name your URL url.
# define radius
radius = 500
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)


In [134]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5deca68fd039930028230705'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lakewood',
  'headerFullLocation': 'Lakewood',
  'headerLocationGranularity': 'city',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 33.853211434500004,
    'lng': -118.10817096183789},
   'sw': {'lat': 33.8442114255, 'lng': -118.11898743816215}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c29187ce19720a1d6f8f858',
       'name': 'California Sushi & Teriyaki',
       'location': {'address': '5916 Del Amo Blvd',
        'crossStreet': 'Woodruff Ave.',
        'lat': 33.84688796644779,
        'lng': -118.11568436500919,
        'labeledLatLngs': [{'label'

In [135]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [136]:
#Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,postalCode,lat,lng
0,California Sushi & Teriyaki,Japanese Restaurant,90713,33.846888,-118.115684
1,Starbucks,Coffee Shop,90713,33.846731,-118.116127
2,Ralphs,Supermarket,90713,33.848014,-118.115557
3,99 Cents Only Stores,Discount Store,90713,33.847387,-118.117002
4,7-Eleven,Convenience Store,90713,33.846561,-118.116743


In [137]:
#And how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


# Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting what type of restaurants are available in each areas of LA County that have low restaurant density. We will limit our analysis to area 500 mtr around each zip code in LA County.
In first step we have collected the required data: location and type (category) of every restaurant in LA County. We have to filter the data from foursqare to select only Restaurants / Food joints. 

Second step in our analysis will be calculation and exploration of 'restaurant density' across different areas of LA

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders.
We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

In [144]:
#2. Explore all Cities in LA County by Zip code
#Let's create a function to repeat the same process to all the Cities in LA
def getNearbyVenues(names, zipcode, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, zipcode, lat, lng in zip(names, zipcode, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            zipcode,
            lat, 
            lng, 
            v['venue']['name'], 
#            v['venue']['location']['address'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Zipcode',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
#                  'Venue address',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [145]:
df_merge.head()

,Zip_Code,Postal_City,Latitude,Longitude,URL,Region
0,90713,Lakewood,33.848711,-118.113579,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
1,91306,Winnetka,34.208404,-118.575940,http://maps.latimes.com/neighborhoods/neighbor...,San Fernando Valley
2,90002,Los Angeles,33.948951,-118.246980,NaN,Los Angeles
3,90506,Torrance,33.885353,-118.326598,http://maps.latimes.com/neighborhoods/neighbor...,South Bay
4,90069,West Hollywood,34.089403,-118.379789,http://maps.latimes.com/neighborhoods/neighbor...,Central L.A.


# Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the number of restaurants in every area candidate:

In [146]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called la_venues.
la_venues = getNearbyVenues(names=df_merge['Postal_City'],
                            zipcode=df_merge['Zip_Code'],
                            latitudes=df_merge['Latitude'],
                            longitudes=df_merge['Longitude']
                                  )

Lakewood
Winnetka
Los Angeles
Torrance
West Hollywood
Westlake Village
Los Angeles
West Hills
Azusa
Pasadena
Lancaster
San Fernando
Alhambra
West Covina
Los Angeles
Los Angeles
Long Beach
Long Beach
Compton
Mission Hills
Glendale
Los Angeles
Tarzana
Covina
Los Angeles
Glendale
South El Monte
Buena Park
La Mirada
Los Angeles
Whittier
Palmdale
Santa Monica
Pearblossom
Granada Hills
Los Angeles
Lomita
Redondo Beach
Torrance
Long Beach
Duarte
Inglewood
Pico Rivera
Los Alamitos
El Segundo
South Pasadena
Los Angeles
Los Angeles
North Hollywood
San Pedro
Glendale
Glendale
Inglewood
Pomona
Los Angeles
Encino
Pasadena
Los Angeles
Edwards
Pasadena
San Dimas
Ontario
Los Angeles
Marina Del Rey
Westlake Village
Lake Hughes
Los Angeles
La Crescenta
Valencia
Venice
Lancaster
Los Angeles
Whittier
Hawaiian Gardens
Los Angeles
Rancho Palos Verdes
Northridge
Long Beach
Pacoima
Monterey Park
Los Angeles
Gardena
Los Angeles
Los Angeles
Montclair
Walnut
Carson
Paramount
North Hollywood
Duarte
El Monte
Palmd

In [199]:
print(la_venues.shape)
la_venues.head()

(6425, 8)


,Neighborhood,Zipcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lakewood,90713,33.848711,-118.113579,California Sushi & Teriyaki,33.846888,-118.115684,Japanese Restaurant
1,Lakewood,90713,33.848711,-118.113579,Starbucks,33.846731,-118.116127,Coffee Shop
2,Lakewood,90713,33.848711,-118.113579,Ralphs,33.848014,-118.115557,Supermarket
3,Lakewood,90713,33.848711,-118.113579,99 Cents Only Stores,33.847387,-118.117002,Discount Store
4,Lakewood,90713,33.848711,-118.113579,7-Eleven,33.846561,-118.116743,Convenience Store


In [200]:
la_venues["Venue Category"].unique()

array(['Japanese Restaurant', 'Coffee Shop', 'Supermarket',
       'Discount Store', 'Convenience Store', 'Video Store',
       'Fast Food Restaurant', 'Wings Joint', 'Music Store', 'Bar',
       'Latin American Restaurant', 'Mexican Restaurant',
       'Filipino Restaurant', 'Ice Cream Shop', 'Grocery Store',
       'South American Restaurant', 'Fried Chicken Joint', 'Food Service',
       "Women's Store", 'Park', 'Mediterranean Restaurant', 'Bookstore',
       'Pizza Place', 'Cosmetics Shop', 'Video Game Store', 'Hookah Bar',
       'Sushi Restaurant', 'Concert Hall', 'Asian Restaurant',
       'Sandwich Place', 'Breakfast Spot', 'Donut Shop',
       'Paper / Office Supplies Store', 'Food', 'Italian Restaurant',
       'Hawaiian Restaurant', 'Restaurant', 'New American Restaurant',
       'Boutique', 'Burger Joint', 'Diner', 'Pet Store', 'Plaza',
       'American Restaurant', 'Health Food Store', 'French Restaurant',
       'Café', 'Gym / Fitness Center', 'Salon / Barbershop', 'Juice

In [201]:
search_values = ['Restaurant','Coffee','Food','Fish','Chips','Pizza','Taco','Café', 'Cafe', 'Bakery','Donut','Pizza','Wings','Hot dog','Cake','Steakhouse','Deli','Soup','Dessert','Bistro','Yogurt','Pie','Noodle','Gourmet', 'Bar',]

In [202]:
#la_venues[la_venues.name.str.contains('|'.join(search_values ))]
la_venues_food = la_venues[la_venues["Venue Category"].str.contains('|'.join(search_values ))]

In [203]:
print(la_venues_food.shape)
la_venues_food.head()

(3043, 8)


,Neighborhood,Zipcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lakewood,90713,33.848711,-118.113579,California Sushi & Teriyaki,33.846888,-118.115684,Japanese Restaurant
1,Lakewood,90713,33.848711,-118.113579,Starbucks,33.846731,-118.116127,Coffee Shop
6,Lakewood,90713,33.848711,-118.113579,McDonald's,33.846284,-118.115973,Fast Food Restaurant
7,Lakewood,90713,33.848711,-118.113579,Taco Bell,33.846726,-118.117500,Fast Food Restaurant
8,Lakewood,90713,33.848711,-118.113579,The Buffalo Spot,33.846601,-118.115791,Wings Joint


In [204]:
#Let's check how many venues were returned in each neighborhoodcity + zip
la_venues_food.groupby('Neighborhood').count()

#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

There are 378 uniques categories.


In [205]:
#Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,California Sushi & Teriyaki,Japanese Restaurant,33.846888,-118.115684
1,Starbucks,Coffee Shop,33.846731,-118.116127
2,Ralphs,Supermarket,33.848014,-118.115557
3,99 Cents Only Stores,Discount Store,33.847387,-118.117002
4,7-Eleven,Convenience Store,33.846561,-118.116743


In [206]:
df_data1 = df_merge
df_data1.head()

,Zip_Code,Postal_City,Latitude,Longitude,URL,Region
0,90713,Lakewood,33.848711,-118.113579,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
1,91306,Winnetka,34.208404,-118.575940,http://maps.latimes.com/neighborhoods/neighbor...,San Fernando Valley
2,90002,Los Angeles,33.948951,-118.246980,NaN,Los Angeles
3,90506,Torrance,33.885353,-118.326598,http://maps.latimes.com/neighborhoods/neighbor...,South Bay
4,90069,West Hollywood,34.089403,-118.379789,http://maps.latimes.com/neighborhoods/neighbor...,Central L.A.


In [207]:
df_data2 = df_data1.rename(columns={"Postal_City": "Neighborhood", "Latitude": "Neighborhood Latitude", "Longitude": "Neighborhood Longitude"})
df_data2.head()

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,URL,Region
0,90713,Lakewood,33.848711,-118.113579,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
1,91306,Winnetka,34.208404,-118.575940,http://maps.latimes.com/neighborhoods/neighbor...,San Fernando Valley
2,90002,Los Angeles,33.948951,-118.246980,NaN,Los Angeles
3,90506,Torrance,33.885353,-118.326598,http://maps.latimes.com/neighborhoods/neighbor...,South Bay
4,90069,West Hollywood,34.089403,-118.379789,http://maps.latimes.com/neighborhoods/neighbor...,Central L.A.


In [208]:
la_venues_mrg = pd.merge(la_venues_food, df_data2, on=['Neighborhood','Neighborhood Latitude', 'Neighborhood Longitude'])
la_venues_mrg.head()

,Neighborhood,Zipcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Zip_Code,URL,Region
0,Lakewood,90713,33.848711,-118.113579,California Sushi & Teriyaki,33.846888,-118.115684,Japanese Restaurant,90713,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
1,Lakewood,90713,33.848711,-118.113579,Starbucks,33.846731,-118.116127,Coffee Shop,90713,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
2,Lakewood,90713,33.848711,-118.113579,McDonald's,33.846284,-118.115973,Fast Food Restaurant,90713,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
3,Lakewood,90713,33.848711,-118.113579,Taco Bell,33.846726,-118.117500,Fast Food Restaurant,90713,http://maps.latimes.com/neighborhoods/neighbor...,Harbor
4,Lakewood,90713,33.848711,-118.113579,The Buffalo Spot,33.846601,-118.115791,Wings Joint,90713,http://maps.latimes.com/neighborhoods/neighbor...,Harbor


In [209]:
#3. ANALYZE EACH NEIGHBORHOOD
# one hot encoding
la_onehot = pd.get_dummies(la_venues_mrg[['Venue Category']], prefix="", prefix_sep="")

# add Neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues_mrg['Neighborhood']
la_onehot['Zip_Code'] = la_venues_mrg['Zip_Code']

# move Neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Zip_Code,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Beer Bar,Bistro,Brazilian Restaurant,Burmese Restaurant,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Cuban Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Frozen Yogurt Shop,Gaming Cafe,Gay Bar,German Restaurant,Gourmet Shop,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Health Food Store,Hookah Bar,Hotel Bar,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mongolian Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Piano Bar,Pie Shop,Pizza Place,Polish Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Salon / Barbershop,Seafood Restaurant,Shabu-Shabu Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tiki Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Neighborhood
0,90713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lakewood
1,90713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lakewood
2,90713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lakewood
3,90713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lakewood
4,90713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Lakewood


In [161]:
# add neighborhood column back to dataframe
la_onehot['Zip_Code'] = la_venues_mrg['Zip_Code'] 
# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Zip_Code,Zoo,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Classroom,College Quad,College Rec Center,College Residence Hall,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pha

In [210]:
la_onehot.shape

(3051, 114)

In [211]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
la_grouped = la_onehot.groupby(['Zip_Code']).mean().reset_index()
la_grouped

,Zip_Code,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Beer Bar,Bistro,Brazilian Restaurant,Burmese Restaurant,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Cuban Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Frozen Yogurt Shop,Gaming Cafe,Gay Bar,German Restaurant,Gourmet Shop,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Health Food Store,Hookah Bar,Hotel Bar,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mongolian Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Piano Bar,Pie Shop,Pizza Place,Polish Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Salon / Barbershop,Seafood Restaurant,Shabu-Shabu Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tiki Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,90001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,90003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,90004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [177]:
#la_grouped['NeighborhoodZip'] = la_grouped['Neighborhood'] + la_grouped['Zip_Code'].map(str)

In [178]:
#la_grouped['NeighborhoodZip'].head()

In [212]:
la_grouped.shape

(308, 113)

In [213]:
#Let's print each neighborhood+ZipCode along with the top 5 most common venues
num_top_venues = 5

for hood in la_grouped['Zip_Code'].astype(str):
    print("----"+hood+"----")
    temp = la_grouped[la_grouped['Zip_Code'].astype(str) == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----90001----
                  venue  freq
0            Donut Shop  0.25
1           Pizza Place  0.25
2  Fast Food Restaurant  0.25
3    Mexican Restaurant  0.25
4              Pie Shop  0.00


----90003----
                             venue  freq
0             Fast Food Restaurant  0.25
1  Southern / Soul Food Restaurant  0.25
2                       Taco Place  0.25
3                      Pizza Place  0.25
4                Afghan Restaurant  0.00


----90004----
                venue  freq
0  Mexican Restaurant  0.33
1        Cocktail Bar  0.33
2    Sushi Restaurant  0.17
3         Pizza Place  0.17
4  Italian Restaurant  0.00


----90005----
                 venue  freq
0    Korean Restaurant  0.45
1           Restaurant  0.09
2          Karaoke Bar  0.09
3     Sushi Restaurant  0.05
4  Japanese Restaurant  0.05


----90006----
                  venue  freq
0     Korean Restaurant  0.67
1           Pizza Place  0.08
2  Fast Food Restaurant  0.08
3           Coffee Shop  0.08
4   

4  American Restaurant  0.08


----90046----
                      venue  freq
0               Coffee Shop  0.23
1           Thai Restaurant  0.15
2      Fast Food Restaurant  0.08
3  Mediterranean Restaurant  0.08
4               Pizza Place  0.08


----90047----
                venue  freq
0  Mexican Restaurant   0.2
1       Deli / Bodega   0.2
2         Coffee Shop   0.2
3          Taco Place   0.2
4  Chinese Restaurant   0.2


----90048----
                venue  freq
0  Mexican Restaurant  0.13
1  Seafood Restaurant  0.10
2           Juice Bar  0.10
3                Café  0.07
4         Pizza Place  0.07


----90049----
                venue  freq
0  Italian Restaurant  0.33
1              Bakery  0.22
2          Restaurant  0.11
3                Café  0.11
4         Coffee Shop  0.11


----90056----
                      venue  freq
0       American Restaurant  0.33
1      Fast Food Restaurant  0.33
2        Mexican Restaurant  0.33
3         Afghan Restaurant  0.00
4  Mediterran

                 venue  freq
0         Noodle House  0.14
1  Peruvian Restaurant  0.14
2          Pizza Place  0.14
3   Chinese Restaurant  0.14
4          Wings Joint  0.07


----90266----
                 venue  freq
0   African Restaurant   1.0
1    Afghan Restaurant   0.0
2             Pie Shop   0.0
3            Piano Bar   0.0
4  Peruvian Restaurant   0.0


----90270----
                venue  freq
0  Mexican Restaurant   0.3
1         Pizza Place   0.2
2              Bakery   0.1
3            Dive Bar   0.1
4  Seafood Restaurant   0.1


----90274----
                      venue  freq
0               Pizza Place  0.22
1                 Juice Bar  0.11
2           Thai Restaurant  0.11
3  Mediterranean Restaurant  0.11
4               Coffee Shop  0.11


----90275----
               venue  freq
0               Food   1.0
1  Afghan Restaurant   0.0
2  Hotpot Restaurant   0.0
3           Pie Shop   0.0
4          Piano Bar   0.0


----90277----
                       venue  freq
0  

                      venue  freq
0     Vietnamese Restaurant  0.33
1                    Bakery  0.33
2        Mexican Restaurant  0.33
3         Afghan Restaurant  0.00
4  Mediterranean Restaurant  0.00


----90670----
               venue  freq
0               Food   1.0
1  Afghan Restaurant   0.0
2  Hotpot Restaurant   0.0
3           Pie Shop   0.0
4          Piano Bar   0.0


----90701----
                   venue  freq
0      Indian Restaurant  0.40
1     Chinese Restaurant  0.20
2  Vietnamese Restaurant  0.08
3            Pizza Place  0.04
4     Frozen Yogurt Shop  0.04


----90703----
                      venue  freq
0                Food Truck   1.0
1         Afghan Restaurant   0.0
2  Mediterranean Restaurant   0.0
3                  Pie Shop   0.0
4                 Piano Bar   0.0


----90704----
                 venue  freq
0   Seafood Restaurant  0.20
1          Pizza Place  0.10
2  American Restaurant  0.10
3                  Bar  0.10
4                 Café  0.05


----

                      venue  freq
0                  Dive Bar   0.1
1        Italian Restaurant   0.1
2  Mediterranean Restaurant   0.1
3          Sushi Restaurant   0.1
4                      Café   0.1


----91024----
                 venue  freq
0          Pizza Place  0.12
1  American Restaurant  0.12
2          Coffee Shop  0.12
3            Juice Bar  0.12
4      Thai Restaurant  0.06


----91030----
                venue  freq
0         Pizza Place  0.18
1  Chinese Restaurant  0.09
2              Bakery  0.09
3         Coffee Shop  0.09
4  Frozen Yogurt Shop  0.09


----91040----
                      venue  freq
0  Mediterranean Restaurant   0.2
1          Asian Restaurant   0.2
2           Thai Restaurant   0.2
3                Donut Shop   0.2
4        Mexican Restaurant   0.2


----91042----
                 venue  freq
0           Restaurant   1.0
1    Afghan Restaurant   0.0
2          Pizza Place   0.0
3            Piano Bar   0.0
4  Peruvian Restaurant   0.0


----91046-

                       venue  freq
0       Fast Food Restaurant  0.33
1                Coffee Shop  0.33
2         Chinese Restaurant  0.33
3          Afghan Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----91343----
                      venue  freq
0      Fast Food Restaurant  0.33
1           Thai Restaurant  0.33
2               Pizza Place  0.33
3         Afghan Restaurant  0.00
4  Mediterranean Restaurant  0.00


----91344----
                       venue  freq
0         Chinese Restaurant  0.14
1                Pizza Place  0.14
2                Coffee Shop  0.14
3                        Bar  0.07
4  Middle Eastern Restaurant  0.07


----91345----
                      venue  freq
0       American Restaurant   0.5
1       Hawaiian Restaurant   0.5
2         Afghan Restaurant   0.0
3  Mediterranean Restaurant   0.0
4                  Pie Shop   0.0


----91346----
                       venue  freq
0       Fast Food Restaurant  0.43
1                Pizza Place  0.14
2  

----91602----
                venue  freq
0          Food Truck  0.17
1  Mexican Restaurant  0.11
2         Pizza Place  0.11
3     Thai Restaurant  0.11
4          Donut Shop  0.06


----91604----
                venue  freq
0         Coffee Shop  0.21
1    Sushi Restaurant  0.17
2  Mexican Restaurant  0.12
3  Seafood Restaurant  0.04
4                Café  0.04


----91605----
                      venue  freq
0                Food Truck   0.5
1        Mexican Restaurant   0.5
2         Afghan Restaurant   0.0
3  Mediterranean Restaurant   0.0
4                  Pie Shop   0.0


----91606----
                     venue  freq
0     Fast Food Restaurant  0.27
1       Mexican Restaurant  0.09
2  New American Restaurant  0.09
3      American Restaurant  0.09
4                   Bakery  0.09


----91607----
                venue  freq
0    Sushi Restaurant  0.11
1     Thai Restaurant  0.11
2    Cuban Restaurant  0.11
3          Food Truck  0.11
4  Chinese Restaurant  0.11


----91608----


                       venue  freq
0        Szechuan Restaurant  0.11
1         Italian Restaurant  0.11
2                Pizza Place  0.11
3  Cajun / Creole Restaurant  0.11
4                 Taco Place  0.11


----92647----
                      venue  freq
0          Asian Restaurant  0.33
1             Deli / Bodega  0.33
2        Mexican Restaurant  0.33
3         Afghan Restaurant  0.00
4  Mediterranean Restaurant  0.00


----92648----
                      venue  freq
0                    Bakery   0.5
1               Coffee Shop   0.5
2         Afghan Restaurant   0.0
3  Mediterranean Restaurant   0.0
4                  Pie Shop   0.0


----92649----
                      venue  freq
0               Pizza Place  0.16
1  Mediterranean Restaurant  0.11
2       American Restaurant  0.11
3          Sushi Restaurant  0.11
4                Restaurant  0.05


----92683----
                      venue  freq
0          Asian Restaurant   0.5
1                       Bar   0.5
2         Af

In [214]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [250]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip_Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zipcode_venues_sorted = pd.DataFrame(columns=columns)
zipcode_venues_sorted['Zip_Code'] = la_grouped['Zip_Code']

for ind in np.arange(la_grouped.shape[0]):
    zipcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

zipcode_venues_sorted.head(10)

,Zip_Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,90001,Pizza Place,Donut Shop,Fast Food Restaurant,Mexican Restaurant,Donburi Restaurant,Dongbei Restaurant,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
1,90003,Pizza Place,Fast Food Restaurant,Taco Place,Southern / Soul Food Restaurant,Fish Market,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
2,90004,Cocktail Bar,Mexican Restaurant,Pizza Place,Sushi Restaurant,Fish Market,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
3,90005,Korean Restaurant,Restaurant,Karaoke Bar,Café,Sushi Restaurant,Mexican Restaurant,Japanese Restaurant,Steakhouse,Noodle House,Halal Restaurant
4,90006,Korean Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
5,90007,Coffee Shop,Food Truck,Korean Restaurant,Caribbean Restaurant,Italian Restaurant,Mexican Restaurant,Mediterranean Restaurant,Café,Eastern European Restaurant,English Restaurant
6,90008,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,New American Restaurant,Pizza Place,Bakery,Southern / Soul Food Restaurant,Seafood Restaurant,Mexican Restaurant,Ethiopian Restaurant
7,90009,Greek Restaurant,Taco Place,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
8,90010,Korean Restaurant,Café,Coffee Shop,Japanese Restaurant,Asian Restaurant,Bakery,Brazilian Restaurant,Sushi Restaurant,Dessert Shop,Ramen Restaurant
9,90011,Fast Food Restaurant,Pizza Place,Restaurant,Donut Shop,Mexican Restaurant,Fish & Chips Shop,Doner Restaurant,Dongbei Restaurant,Dumpling Restaurant,Eastern European Restaurant


In [251]:
#4. Cluster Neighborhoods
#Run k-means to cluster the neighborhood into 10 clusters.
# set number of clusters
kclusters = 10

la_grouped_clustering = la_grouped.drop('Zip_Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([9, 9, 1, 5, 5, 3, 9, 1, 1, 9], dtype=int32)

In [252]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
zipcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

la_merged = df_data2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(zipcode_venues_sorted.set_index('Zip_Code'), on='Zip_Code')

la_merged.head() # check the last columns!

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,URL,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,90713,Lakewood,33.848711,-118.113579,http://maps.latimes.com/neighborhoods/neighbor...,Harbor,9.0,Fast Food Restaurant,Wings Joint,Japanese Restaurant,Coffee Shop,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
1,91306,Winnetka,34.208404,-118.575940,http://maps.latimes.com/neighborhoods/neighbor...,San Fernando Valley,1.0,Latin American Restaurant,Bar,Mexican Restaurant,Food Service,South American Restaurant,Filipino Restaurant,Wings Joint,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
2,90002,Los Angeles,33.948951,-118.246980,NaN,Los Angeles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90506,Torrance,33.885353,-118.326598,http://maps.latimes.com/neighborhoods/neighbor...,South Bay,9.0,Fast Food Restaurant,Pizza Place,Mediterranean Restaurant,Mexican Restaurant,Hookah Bar,Sushi Restaurant,Italian Restaurant,Food,Donut Shop,Restaurant
4,90069,West Hollywood,34.089403,-118.379789,http://maps.latimes.com/neighborhoods/neighbor...,Central L.A.,1.0,New American Restaurant,Sushi Restaurant,Coffee Shop,Salon / Barbershop,Café,French Restaurant,Chinese Restaurant,Restaurant,Italian Restaurant,Juice Bar


In [253]:
la_merged = la_merged[pd.notnull(la_merged['Cluster Labels'])]

In [254]:
la_merged['Cluster Labels'] =la_merged['Cluster Labels'].astype(int)

In [255]:
la_merged = la_merged.drop(columns=['URL'])

In [256]:
la_merged.head()

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,90713,Lakewood,33.848711,-118.113579,Harbor,9,Fast Food Restaurant,Wings Joint,Japanese Restaurant,Coffee Shop,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
1,91306,Winnetka,34.208404,-118.575940,San Fernando Valley,1,Latin American Restaurant,Bar,Mexican Restaurant,Food Service,South American Restaurant,Filipino Restaurant,Wings Joint,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
3,90506,Torrance,33.885353,-118.326598,South Bay,9,Fast Food Restaurant,Pizza Place,Mediterranean Restaurant,Mexican Restaurant,Hookah Bar,Sushi Restaurant,Italian Restaurant,Food,Donut Shop,Restaurant
4,90069,West Hollywood,34.089403,-118.379789,Central L.A.,1,New American Restaurant,Sushi Restaurant,Coffee Shop,Salon / Barbershop,Café,French Restaurant,Chinese Restaurant,Restaurant,Italian Restaurant,Juice Bar
5,91361,Westlake Village,34.151380,-118.835697,Santa Monica Mountains,1,Italian Restaurant,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [268]:
#Finally let's visualize the result
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, zipc, cluster in zip(la_merged['Neighborhood Latitude'], la_merged['Neighborhood Longitude'], la_merged['Neighborhood'], la_merged['Zip_Code'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

In [269]:
#Cluster 1
la_merged.loc[la_merged['Cluster Labels'] == 0, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
234,91042,Tujunga,34.254600,-118.287258,Verdugos,0,Restaurant,Wings Joint,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
306,91710,Chino,34.017832,-117.689437,Chino,0,Restaurant,American Restaurant,Bakery,Mexican Restaurant,Wings Joint,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
328,91307,West Hills,34.197811,-118.638008,San Fernando Valley,0,Restaurant,Wings Joint,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
347,90077,Los Angeles,34.104093,-118.451017,Los Angeles,0,Restaurant,Bakery,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant


In [270]:
#Cluster 2
la_merged.loc[la_merged['Cluster Labels'] == 1, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,91306,Winnetka,34.208404,-118.575940,San Fernando Valley,1,Latin American Restaurant,Bar,Mexican Restaurant,Food Service,South American Restaurant,Filipino Restaurant,Wings Joint,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
4,90069,West Hollywood,34.089403,-118.379789,Central L.A.,1,New American Restaurant,Sushi Restaurant,Coffee Shop,Salon / Barbershop,Café,French Restaurant,Chinese Restaurant,Restaurant,Italian Restaurant,Juice Bar
5,91361,Westlake Village,34.151380,-118.835697,Santa Monica Mountains,1,Italian Restaurant,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
6,90064,Los Angeles,34.035412,-118.432177,Los Angeles,1,Japanese Restaurant,Mexican Restaurant,Brazilian Restaurant,Sports Bar,Pizza Place,Indian Restaurant,Bar,Chinese Restaurant,Salon / Barbershop,Filipino Restaurant
9,91104,Pasadena,34.167000,-118.127588,Verdugos,1,Wings Joint,Bakery,Chinese Restaurant,Food,Latin American Restaurant,Italian Restaurant,Donut Shop,Gay Bar,German Restaurant,Dumpling Restaurant
10,93535,Lancaster,34.688762,-118.059288,Antelope Valley,1,Bar,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
12,91803,Alhambra,34.075334,-118.144307,San Gabriel Valley,1,Mexican Restaurant,Asian Restaurant,Italian Restaurant,Szechuan Restaurant,Taco Place,Indonesian Restaurant,Cajun / Creole Restaurant,Pizza Place,Vietnamese Restaurant,Eastern European Restaurant
20,91210,Glendale,34.144311,-118.256108,Verdugos,1,Café,New American Restaurant,Pie Shop,Ramen Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Italian Restaurant,Dumpling Restaurant,Pizza Place
22,91356,Tarzana,34.169903,-118.540489,San Fernando Valley,1,Japanese Restaurant,Mexican Restaurant,Deli / Bodega,Italian Restaurant,Brazilian Restaurant,Greek Restaurant,Pizza Place,Chinese Restaurant,Seafood Restaurant,Indian Restaurant
24,90035,Los Angeles,34.054282,-118.383856,Los Angeles,1,Food Truck,Kosher Restaurant,Thai Restaurant,French Restaurant,Italian Restaurant,Coffee Shop,Japanese Restaurant,Deli / Bodega,Steakhouse,Sushi Restaurant


In [271]:
#Cluster 3
la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,90275,Rancho Palos Verdes,33.763102,-118.366658,South Bay,2,Food,Wings Joint,Wine Bar,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
227,90670,Santa Fe Springs,33.935920,-118.066088,Southeast,2,Food,Wings Joint,Wine Bar,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
239,90745,Carson,33.825974,-118.269669,Harbor,2,Bakery,Food,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
240,91775,San Gabriel,34.115651,-118.085736,San Gabriel Valley,2,Food,Wings Joint,Wine Bar,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
277,90058,Los Angeles,34.001842,-118.221127,Los Angeles,2,Food,Food Truck,Wings Joint,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
331,90710,Harbor City,33.794803,-118.298729,Harbor,2,Food,Mexican Restaurant,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [272]:
#Cluster 4
la_merged.loc[la_merged['Cluster Labels'] == 3, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,90840,Long Beach,33.775342,-118.113466,Harbor,3,Coffee Shop,Food Court,Wings Joint,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
23,91724,Covina,34.091232,-117.858336,San Gabriel Valley,3,Coffee Shop,Wings Joint,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
59,91125,Pasadena,34.135912,-118.125758,Verdugos,3,Café,Restaurant,Coffee Shop,Food Truck,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
77,90808,Long Beach,33.824962,-118.115317,Harbor,3,Café,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant
100,91416,Encino,34.176112,-118.518738,San Fernando Valley,3,Food Truck,Coffee Shop,Wings Joint,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
102,90021,Los Angeles,34.030124,-118.241779,Los Angeles,3,Food Truck,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Hawaiian Restaurant,Café,Asian Restaurant,Mexican Restaurant,Seafood Restaurant,Wings Joint
111,91523,Burbank,34.156433,-118.333298,San Fernando Valley,3,Food Truck,Fast Food Restaurant,American Restaurant,Steakhouse,Mexican Restaurant,Karaoke Bar,Coffee Shop,Wings Joint,Filipino Restaurant,Falafel Restaurant
128,91046,Verdugo City,34.211302,-118.238598,Verdugo City,3,Café,Coffee Shop,Mexican Restaurant,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
149,90623,La Palma,33.849641,-118.039988,La Palma,3,Coffee Shop,Wings Joint,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
165,90094,Playa Vista,33.972931,-118.425329,Westside,3,Food Truck,Café,Asian Restaurant,Fast Food Restaurant,Coffee Shop,Pizza Place,German Restaurant,Fish & Chips Shop,Donut Shop,Dumpling Restaurant


In [273]:
#Cluster 5
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,90018,Los Angeles,34.030772,-118.316869,Los Angeles,4,Bakery,Taco Place,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
16,90822,Long Beach,33.798314,-118.151368,Harbor,4,Wine Bar,Bakery,Food & Drink Shop,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
78,91334,Pacoima,34.241604,-118.425616,San Fernando Valley,4,Bakery,Food Truck,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
136,90221,Compton,33.895603,-118.205637,Southeast,4,Bakery,Chinese Restaurant,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
177,92648,Huntington Beach,33.675663,-117.999727,Huntington Beach,4,Bakery,Coffee Shop,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
186,90631,La Habra,33.931801,-117.952658,La Habra,4,Bakery,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
271,90044,Los Angeles,33.953861,-118.291946,Los Angeles,4,Wine Bar,Wings Joint,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
287,91744,La Puente,34.030754,-117.939666,San Gabriel Valley,4,Bakery,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant


In [274]:
#Cluster 6
la_merged.loc[la_merged['Cluster Labels'] == 5, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,90006,Los Angeles,34.050102,-118.292689,Los Angeles,5,Korean Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
67,91224,La Crescenta,34.231311,-118.251477,La Crescenta,5,Korean Restaurant,Chinese Restaurant,Coffee Shop,Donut Shop,Asian Restaurant,Café,Indian Restaurant,Juice Bar,Mexican Restaurant,Fish & Chips Shop
80,90020,Los Angeles,34.066361,-118.301648,Los Angeles,5,Korean Restaurant,Café,Dessert Shop,Shabu-Shabu Restaurant,Seafood Restaurant,Salon / Barbershop,Karaoke Bar,Japanese Restaurant,Restaurant,Coffee Shop
95,92833,Fullerton,33.870154,-117.963007,Fullerton,5,Korean Restaurant,Japanese Restaurant,Donut Shop,Pizza Place,Wine Bar,Gourmet Shop,Fish Market,Dongbei Restaurant,Dumpling Restaurant,Eastern European Restaurant
120,91320,Newbury Park,34.181361,-118.935389,Newbury Park,5,Korean Restaurant,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
163,91768,Pomona,34.063963,-117.778996,Pomona Valley,5,Korean Restaurant,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
199,90005,Los Angeles,34.059071,-118.303017,Los Angeles,5,Korean Restaurant,Restaurant,Karaoke Bar,Café,Sushi Restaurant,Mexican Restaurant,Japanese Restaurant,Steakhouse,Noodle House,Halal Restaurant
235,91214,La Crescenta,34.228574,-118.246949,La Crescenta,5,Korean Restaurant,Juice Bar,Kebab Restaurant,Chinese Restaurant,Wings Joint,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
310,90715,Lakewood,33.839661,-118.079549,Harbor,5,Korean Restaurant,Pizza Place,Salon / Barbershop,Japanese Restaurant,Wings Joint,Fish Market,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
334,90621,Buena Park,33.870371,-117.996990,Buena Park,5,Korean Restaurant,Café,Fast Food Restaurant,Mexican Restaurant,Food,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant


In [275]:
#Cluster 7
la_merged.loc[la_merged['Cluster Labels'] == 6, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,91702,Azusa,34.127122,-117.906210,San Gabriel Valley,6,Mexican Restaurant,Taco Place,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
26,91733,South El Monte,34.055104,-118.046996,San Gabriel Valley,6,Mexican Restaurant,Wings Joint,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
30,90602,Whittier,33.971141,-118.035056,San Gabriel Valley,6,Mexican Restaurant,Wings Joint,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
41,90304,Inglewood,33.939383,-118.358129,South Bay,6,Mexican Restaurant,Pizza Place,Taco Place,Latin American Restaurant,Wings Joint,Fish & Chips Shop,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
44,90245,El Segundo,33.920152,-118.405807,South Bay,6,Food Truck,Mexican Restaurant,Wings Joint,Fish Market,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
61,91762,Ontario,34.057793,-117.663289,Ontario,6,Mexican Restaurant,Wings Joint,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
73,90716,Hawaiian Gardens,33.831371,-118.071446,Harbor,6,Mexican Restaurant,Wings Joint,Peruvian Restaurant,Pizza Place,Noodle House,Coffee Shop,Fast Food Restaurant,Korean Restaurant,Persian Restaurant,Food Service
88,91605,North Hollywood,34.205462,-118.401940,San Fernando Valley,6,Food Truck,Mexican Restaurant,Wings Joint,Fish Market,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
103,91722,Covina,34.095803,-117.905810,San Gabriel Valley,6,Mexican Restaurant,Fast Food Restaurant,Dive Bar,Coffee Shop,Wings Joint,Fish Market,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
112,90248,Gardena,33.875442,-118.282378,South Bay,6,Mexican Restaurant,Wings Joint,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [276]:
#Cluster 8
la_merged.loc[la_merged['Cluster Labels'] == 7, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,91208,Glendale,34.190020,-118.227166,Verdugos,7,Middle Eastern Restaurant,Wings Joint,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
349,91203,Glendale,34.152823,-118.261327,Verdugos,7,Middle Eastern Restaurant,Wings Joint,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [277]:
#Cluster 9
la_merged.loc[la_merged['Cluster Labels'] == 8, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,91345,San Fernando,34.262121,-118.461758,San Fernando Valley,8,Hawaiian Restaurant,American Restaurant,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
76,91325,Northridge,34.235882,-118.521058,San Fernando Valley,8,American Restaurant,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
105,91108,San Marino,34.120983,-118.113136,San Gabriel Valley,8,American Restaurant,Sushi Restaurant,Wings Joint,Food,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
295,91502,Burbank,34.176442,-118.306476,San Fernando Valley,8,Café,American Restaurant,Asian Restaurant,Steakhouse,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
318,91118,San Marino,34.126544,-118.091928,San Gabriel Valley,8,American Restaurant,Asian Restaurant,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
362,90639,La Mirada,33.905544,-118.018020,Southeast,8,American Restaurant,College Cafeteria,Wings Joint,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [278]:
#Cluster 10
la_merged.loc[la_merged['Cluster Labels'] == 9, la_merged.columns[[0] + list(range(1, la_merged.shape[1]))]]

,Zip_Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Region,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,90713,Lakewood,33.848711,-118.113579,Harbor,9,Fast Food Restaurant,Wings Joint,Japanese Restaurant,Coffee Shop,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
3,90506,Torrance,33.885353,-118.326598,South Bay,9,Fast Food Restaurant,Pizza Place,Mediterranean Restaurant,Mexican Restaurant,Hookah Bar,Sushi Restaurant,Italian Restaurant,Food,Donut Shop,Restaurant
19,91346,Mission Hills,34.267844,-118.467189,San Fernando Valley,9,Fast Food Restaurant,Pizza Place,Bar,Chinese Restaurant,Latin American Restaurant,Food,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
21,90037,Los Angeles,34.003494,-118.287097,Los Angeles,9,Fast Food Restaurant,Food,Bakery,Food Truck,Chinese Restaurant,Wings Joint,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant
27,90620,Buena Park,33.842780,-118.012349,Buena Park,9,Latin American Restaurant,Fast Food Restaurant,Health Food Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
28,90638,La Mirada,33.902672,-118.007087,Southeast,9,Steakhouse,Bakery,Coffee Shop,Fast Food Restaurant,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant
43,90720,Los Alamitos,33.798952,-118.068200,Los Alamitos,9,Fast Food Restaurant,Food Truck,Pie Shop,American Restaurant,Asian Restaurant,Bakery,Bar,Indian Chinese Restaurant,Sports Bar,Steakhouse
46,90031,Los Angeles,34.077524,-118.213106,Los Angeles,9,Mexican Restaurant,Fast Food Restaurant,Juice Bar,Taco Place,Seafood Restaurant,Pizza Place,Bakery,Wings Joint,Falafel Restaurant,English Restaurant
48,91606,North Hollywood,34.186731,-118.386806,San Fernando Valley,9,Fast Food Restaurant,Mexican Restaurant,American Restaurant,New American Restaurant,Deli / Bodega,Thai Restaurant,Bakery,Food Truck,Taco Place,Wings Joint
60,91773,San Dimas,34.103164,-117.814838,San Gabriel Valley,9,Mexican Restaurant,Fast Food Restaurant,Sushi Restaurant,Vietnamese Restaurant,American Restaurant,Pizza Place,Thai Restaurant,Wings Joint,Falafel Restaurant,Ethiopian Restaurant


# Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Los  Angeles County , there are pockets of low restaurant density towwards the north and eastern parts of the county. Highest concentration of restaurants was detected in Los Angeles (downtown) and southern part. 

We were able to distintly identfy the clusters where cwertain kinds of cuisinesare popular predominantly (for example for Mexican, Korean, Bakery). This will help us  determine some location to open a certain kind of cuisine. Of course, it does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas within LA but not crowded with existing restaurants   Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.